# Written Summer 2019. Refactored in summer 2021, damn my code was bad. 
> Whether you should buy above the below the SMA

Note `df2 = dataset.iloc[ : :20, :] # every 20th, all columns`, this is on a (apx.) monthly basis, as there are (apx.) 20 trading days a month... use `consider_every_Nth_day`

In [1]:
consider_every_Nth_day = 1 # 20 
share = 'SPY'

In [2]:
# !pip install tensorflow
# !pip install pandas-datareader
# !pip install yfinance

In [3]:
# Importing the libraries
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

from pandas_datareader import data as pdr
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

In [4]:
dataset = yf.download(share,"2003-3-14") #, start="2017-01-01"
dataset = dataset.drop(columns=['Open','High', 'Low','Adj Close','Volume'])
dataset.head()

[*********************100%***********************]  1 of 1 completed


,Close
Date,
2003-03-14,84.129997
2003-03-17,86.779999
2003-03-18,87.290001
2003-03-19,87.959999
2003-03-20,88.150002


In [5]:
df2 = dataset.iloc[ : :consider_every_Nth_day, :] # every 20th, all columns
df2.head()

,Close
Date,
2003-03-14,84.129997
2003-03-17,86.779999
2003-03-18,87.290001
2003-03-19,87.959999
2003-03-20,88.150002


In [6]:
invPerMonth = 1000
total =0
shares=0 

for i, row in df2.iterrows():
   # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
    numShares = math.floor((invPerMonth)/row.Close.round(2))
    shares+=numShares
    invPerMonth -= (row.Close.round(2) * numShares)
    
    invPerMonth += 1000
   # print("numShares:" , numShares)
   # print("shares:" , shares)
   # print("invPerMonth:" , invPerMonth)

base_strategy = int(dataset.tail(1).Close*shares)
value_of_shares_at_end = dataset.tail(1).Close.values[0]*shares
print("invPerMonth: ",invPerMonth)
print(f"Numb of shares:{shares}")
print(f"Numb of shares * last closing = {value_of_shares_at_end:.2f}")

invPerMonth:  1301.7299999999927
Numb of shares:29900
Numb of shares * last closing = 12481154.04


In [7]:
# Note the odd usage
def simulate_buy_below_MA(window=1):
    df2['MA'] = df2.rolling(window=window).mean()
    
    invPerMonth = 1000
    total =0
    shares=0 
    for i, row in df2.iterrows():
       # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
        if((row.Close.round(2) <= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
          numb_new_shares = math.floor((invPerMonth)/row.Close.round(2))
          shares += numb_new_shares
          invPerMonth -= (row.Close.round(2) * numb_new_shares)
          #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
          #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
        invPerMonth += 1000
       # print("numb_new_shares:" , numb_new_shares)
       # print("shares:" , shares)
       # print("invPerMonth:" , invPerMonth)
    value_of_shares_at_end = dataset.tail(1).Close.values[0]*shares
    print(f'Window size is {window}')
    print(f"InvPerMonth: {invPerMonth:.2f}")
    print(f"Numb of shares:{shares}")
    print(f"Numb of shares * last closing = {value_of_shares_at_end:.2f}")
    print(f"Total: {value_of_shares_at_end + invPerMonth:.2f}")
    print(f'Total with base_strategy: {base_strategy:.2f}')
    print(f'Improvement over base_strategy: {value_of_shares_at_end + invPerMonth - base_strategy:.2f}')
    print(f'===================================\n')

In [8]:
simulate_buy_below_MA(0)
simulate_buy_below_MA(1)
simulate_buy_below_MA(10)
simulate_buy_below_MA(20)
simulate_buy_below_MA(30)
simulate_buy_below_MA(45)
simulate_buy_below_MA(90)
simulate_buy_below_MA(180)
simulate_buy_below_MA(360)
simulate_buy_below_MA(720)

Window size is 0
InvPerMonth: 4563000.00
Numb of shares:0
Numb of shares * last closing = 0.00
Total: 4563000.00
Total with base_strategy: 12481154.00
Improvement over base_strategy: -7918154.00

Window size is 1
InvPerMonth: 1301.73
Numb of shares:29900
Numb of shares * last closing = 12481154.04
Total: 12482455.77
Total with base_strategy: 12481154.00
Improvement over base_strategy: 1301.77

Window size is 10
InvPerMonth: 4379.32
Numb of shares:29904
Numb of shares * last closing = 12482823.76
Total: 12487203.08
Total with base_strategy: 12481154.00
Improvement over base_strategy: 6049.08

Window size is 20
InvPerMonth: 24048.68
Numb of shares:29813
Numb of shares * last closing = 12444837.64
Total: 12468886.32
Total with base_strategy: 12481154.00
Improvement over base_strategy: -12267.68

Window size is 30
InvPerMonth: 24274.90
Numb of shares:29754
Numb of shares * last closing = 12420209.28
Total: 12444484.18
Total with base_strategy: 12481154.00
Improvement over base_strategy: -3

In [9]:
def simulate_buy_above_MA(window=1):
    df2['MA'] = df2.rolling(window=window).mean()
    
    invPerMonth = 1000
    total =0
    shares=0 
    for i, row in df2.iterrows():
       # print("\n" + str(i) +" | "+ str(row.Close.round(2)))
        if((row.Close.round(2) >= row.MA.round(2)) and not math.isnan(row.MA.round(2))):
          numb_new_shares = math.floor((invPerMonth)/row.Close.round(2))
          shares += numb_new_shares
          invPerMonth -= (row.Close.round(2) * numb_new_shares)
          #print("\n close: " + str(i) +" | "+ str(row.Close.round(2)))
          #print(" MA:    "+str(i) +" | "+ str(row.MA.round(2)))
        invPerMonth += 1000
       # print("numb_new_shares:" , numb_new_shares)
       # print("shares:" , shares)
       # print("invPerMonth:" , invPerMonth)
    value_of_shares_at_end = dataset.tail(1).Close.values[0]*shares

    print(f'Window size is {window}')
    print(f"InvPerMonth: {invPerMonth:.2f}")
    print(f"Numb of shares:{shares}")
    print(f"Numb of shares * last closing = {value_of_shares_at_end:.2f}") #dataset.tail, not df2.tail
    print(f"Total: {value_of_shares_at_end + invPerMonth:.2f}")
    print(f'Total with base_strategy: {base_strategy:.2f}')
    print(f'Improvement over base_strategy: {value_of_shares_at_end + invPerMonth - base_strategy:.2f}')
    print(f'===================================\n')

In [10]:
simulate_buy_above_MA(0)
simulate_buy_above_MA(1)
simulate_buy_above_MA(10)
simulate_buy_above_MA(20)
simulate_buy_above_MA(30)
simulate_buy_above_MA(45)
simulate_buy_above_MA(90)
simulate_buy_above_MA(180)
simulate_buy_above_MA(360)
simulate_buy_above_MA(720)

Window size is 0
InvPerMonth: 4563000.00
Numb of shares:0
Numb of shares * last closing = 0.00
Total: 4563000.00
Total with base_strategy: 12481154.00
Improvement over base_strategy: -7918154.00

Window size is 1
InvPerMonth: 1301.73
Numb of shares:29900
Numb of shares * last closing = 12481154.04
Total: 12482455.77
Total with base_strategy: 12481154.00
Improvement over base_strategy: 1301.77

Window size is 10
InvPerMonth: 1157.31
Numb of shares:29843
Numb of shares * last closing = 12457360.54
Total: 12458517.85
Total with base_strategy: 12481154.00
Improvement over base_strategy: -22636.15

Window size is 20
InvPerMonth: 1022.19
Numb of shares:29834
Numb of shares * last closing = 12453603.67
Total: 12454625.86
Total with base_strategy: 12481154.00
Improvement over base_strategy: -26528.14

Window size is 30
InvPerMonth: 1406.85
Numb of shares:29763
Numb of shares * last closing = 12423966.15
Total: 12425373.00
Total with base_strategy: 12481154.00
Improvement over base_strategy: -5